In [1]:
import sys 
import csv
import numpy as np
import random as rd
import copy

In [2]:
#Change this in actual file to take command line arguments
file1 = 'bitbootcamp/IvyLeague.txt'
file2 = 'bitbootcamp/MajorityRule.txt'

In [3]:
dicts = []

reader = csv.DictReader(open(file1, 'rb'),delimiter='\t')
for line in reader: dicts.append(line)

In [4]:
dicts

[{'CLASS': 'true',
  'GoodGrades': 'true',
  'GoodLetters': 'true',
  'GoodSAT': 'true',
  'HasScholarship': 'true',
  'IsRich': 'true',
  'ParentAlum': 'true',
  'SchoolActivities': 'true'},
 {'CLASS': 'true',
  'GoodGrades': 'true',
  'GoodLetters': 'true',
  'GoodSAT': 'true',
  'HasScholarship': 'true',
  'IsRich': 'true',
  'ParentAlum': 'true',
  'SchoolActivities': 'false'},
 {'CLASS': 'true',
  'GoodGrades': 'true',
  'GoodLetters': 'true',
  'GoodSAT': 'true',
  'HasScholarship': 'true',
  'IsRich': 'true',
  'ParentAlum': 'false',
  'SchoolActivities': 'true'},
 {'CLASS': 'true',
  'GoodGrades': 'true',
  'GoodLetters': 'true',
  'GoodSAT': 'true',
  'HasScholarship': 'true',
  'IsRich': 'true',
  'ParentAlum': 'false',
  'SchoolActivities': 'false'},
 {'CLASS': 'true',
  'GoodGrades': 'true',
  'GoodLetters': 'true',
  'GoodSAT': 'true',
  'HasScholarship': 'false',
  'IsRich': 'true',
  'ParentAlum': 'true',
  'SchoolActivities': 'true'},
 {'CLASS': 'true',
  'GoodGrades': 

In [5]:
#The field labeled by the concept
label = 'CLASS'

def split(dict_list):
    
    ##REMOVE IN PRACTICE
    results = {}
    ####
    
    fields = dict_list[0].keys()
    fields.remove(label)
    best_field = fields[0]
    best_gain = -10000
    num_total = len(dict_list)
    for field in fields:
        temp_dic = {'true':{'true':0,'false':0},'false':{'true':0,'false':0}}
        for ex in dict_list:
            temp_dic[ex[field]][ex[label]] += 1
        
        TT = temp_dic['true']['true']
        TF = temp_dic['true']['false']
        FT = temp_dic['false']['true']
        FF = temp_dic['false']['false']
        
        #Solve issues with division by 0 and log 0 for those classes that are not in the data set
        if (TT==0):
            p11 = 1
        else:
            p11 = float(TT)/(TT+TF)
        if (TF==0):
            p10 = 1
        else:
            p10 = float(TF)/(TT+TF)
        if (FT==0):
            p01=1
        else:
            p01 = float(FT)/(FT+FF)
        if (FF==0):
            p00=1
        else:
            p00 = float(FF)/(FT+FF)
        
        gain = (float(TT+TF)/num_total)*(p11*np.log2(p11)+p10*np.log2(p10))+(float(FT+FF)/num_total)*(p01*np.log2(p01)+p00*np.log2(p00))
        
        ###REMOVE IN PRACTICE
        results[field] = gain
        #######################
        
        if gain > best_gain:
            best_gain = gain
            best_field = field
    
    ###REMOVE IN PRACTICE
    #print results
    #######################
    return best_field

In [6]:
#We will represent the decision tree as an array. The first element of the array will be the spliting criteria,
#the second the set of examples of the tree, the third the left branch (i.e. the false), and the last the right branch(i.e. the true)

In [7]:
#Takes in a list of examples with their attributes, and returns a tree


#Maybe I should make the leaves not lists?

def make_tree(ex_list):
    num_true = 0
    num_false = 0
    for ex in ex_list:
        if ex['CLASS'] == 'true':
            num_true+=1
        else:
            num_false+=1
            
    #If all examples are false
    if num_true == 0:
        return [0]
    #If all examples are true
    if num_false == 0:
        return [1]
    #Conditional at end of function ensures both not to be true
    
    #Record whether there are more true or false
    if num_true >= num_false:
        max_num = 1
    else:
        max_num = 0
        
    #If there are no more attributes to split on
    if len(ex_list[0]) == 1:
        return [max_num]
        
    #Find the splitting criterion
    split_class = split(ex_list)
    
    trues = []
    falses = []
    
    
    #Divide your remaining examples between branches
    for ex in ex_list:
        
        #Stop from changing original argument
        ex2 = ex.copy()
        
        branch = ex2.pop(split_class)
        if branch == 'false':
            falses.append(ex2)
        else:
            trues.append(ex2)
    
    #Ensures that algorithm will receive non-empty lists of examples
    #By instead putting the most frequent classification of the node if a branch is empty
    if len(trues) == 0:
        left_branch = [max_num]
    else:
        left_branch = make_tree(falses)
    if len(falses) == 0:
        right_branch = [max_num]
    else:
        right_branch = make_tree(trues)
        
    return [split_class, left_branch,right_branch]

In [8]:
dic = {1:1,2:2}
print dic.pop(1)
print dic

1
{2: 2}


In [9]:
split(dicts)

'IsRich'

In [10]:
make_tree(dicts)

['IsRich',
 ['HasScholarship', [0], ['GoodSAT', [0], ['GoodLetters', [0], [1]]]],
 ['GoodLetters',
  ['GoodGrades', [0], ['SchoolActivities', [0], [1]]],
  ['GoodSAT', ['GoodGrades', [0], [1]], [1]]]]

In [11]:
def classify(ex,tree):
    attr = tree[0]
    if attr == 0:
        return 'false'
    if attr == 1:
        return 'true'
    if ex[attr]=='false':
        return classify(ex,tree[1])
    return classify(ex,tree[2])
        

In [12]:
tree = make_tree(dicts)
l = []
for item in dicts:
    l.append(classify(item,tree))
l2 = map(lambda x:x['CLASS'],dicts)
num_same=0
num_diff=0
for i in range(len(l)):
    if l[i]==l2[i]:
        num_same+=1
    else:
        num_diff+=0
print num_same
print num_diff

62
0


In [13]:
dicts2 = copy.copy(dicts)
rd.shuffle(dicts2)
tree2 = make_tree(dicts2[:40])
l = []
for item in dicts2:
    l.append(classify(item,tree2))
l2 = map(lambda x:x['CLASS'],dicts2)
num_same=0
num_diff=0
for i in range(len(l)):
    if l[i]==l2[i]:
        num_same+=1
    else:
        num_diff+=1
print num_same
print num_diff

61
1


In [18]:
#For some reason, the tabs don't seem to be of uniform length, so I changed to 3 spaces
def print_tree(tree, parent):
    output = 'parent: '
    output += parent
    output += '   '
    if tree[0] == 0 or tree[0] == 1:
        print output+'-'
    else:
        #Add attribute of node
        output+='attribute: '
        output+=tree[0]
        output+='   '
        
        #Add left(false) child
        output+='falseChild: '
        if tree[1][0]==0:
            output+='falseLeaf'
        elif tree[1][0]==1:
            output+='trueLeaf'
        else:
            output+=tree[1][0]
        output+='   '
        
        #Add right(true) child
        output+='trueChild: '
        if tree[2][0]==0: 
            output+='falseLeaf'
        elif tree[2][0]==1:
            output+='trueLeaf'
        else:
            output+=tree[2][0]
        output+='   '
        
        print output
        
        #Continue printing the tree so long as you are not already a leaf
        print_tree(tree[1],tree[0])
        print_tree(tree[2],tree[0])

In [15]:
########May want to remove some arguments later and use globals############
#############Change trial_number to global if possible###########################
def perform_trials(trial_number, num_trials, train_size, verbose, total_tree, total_prior):
    
    if trial_number>num_trials:
        print 'example file used = '+file1
        print 'number of trials = ',num_trials
        print 'training set size for each trial = ',train_size
        print 'testing set size for each trial = ',len(dicts)-train_size
        print 'mean performance of decision tree over all trials = ',int(round(float(total_tree)/num_trials,0)),'% correct classification'
        print 'mean performance of using prior probability derived from the training set = ',int(round(float(total_prior)/num_trials,0)),'% correct classification'
        return
    else:
        print 'Trial number ',trial_number
        print '-----------------'
        print '\n'
        
        dicts2 = copy.deepcopy(dicts)
        rd.shuffle(dicts2)
        
        train = dicts2[:train_size]
        test = dicts2[train_size:]

        #Computing prior probability
        
        prior_true = float(len([x for x in dicts2 if x['CLASS']=='true']))/len(dicts2)
        if prior_true < 0.5:
            best = 'false'
        else: 
            best = 'true'
            
        tree2 = make_tree(train)
        print 'Decision tree structure: '
        print_tree(tree2,'root')
        print '\n'
        
        if verbose == 1:
            #Print the training set
            print 'Train set: '
            print '-----------'
            print '\n'
            for i,item in enumerate(train):
                print 'Training Example ',i+1
                print '-----------------------' 
                for attr in item.keys():
                    print attr, ': ', item[attr]
                print '\n'
            print '\n'

            #Start printing the testing set
            print 'Test set: '
            print '----------'
            print '\n'
        
        #Store number of correctly classified
        num_correct = 0
        #Store number of prior prob correctly classified
        num_prob = 0
        
        for i,item in enumerate(test):
            item_class = classify(item, tree2)
            if verbose==1:
                #Finish printing the testing set
                print 'Testing Example ',i+1
                print '----------------------' 
                for attr in item.keys():
                    print attr, ': ', item[attr]
                print 'Decision tree classification: ', item_class
                print 'Prior probability classification: ', best
                print '\n'
                
            if item_class == item['CLASS']:
                num_correct+=1
            if best == item['CLASS']:
                num_prob+=1
        
        #Percent correct of each classification used on test cases
        tree_percent = float(100*num_correct)/len(test)
        prior_percent = float(100*num_prob)/len(test)
        
        #Separate from the percentages to come
        if verbose==1:
            print '\n'
        
        print 'Percentage of test cases correctly classified by a decision tree built with ID3 = ', int(round(tree_percent,0)),'%'
        print 'Percentage of test cases correctly classified by prior probabilities from the training set = ', int(round(prior_percent,0)),'%'
        
        #Just putting some space between consecutive trials
        print '\n'
        print '\n'
        print '\n'
            
    return perform_trials(trial_number+1, num_trials,train_size,verbose, total_tree+tree_percent,total_prior+prior_percent)

In [19]:
perform_trials(1,2,20,0,0,0)

Trial number  1
-----------------


Decision tree structure: 
parent: root   attribute: IsRich   falseChild: HasScholarship   trueChild: GoodLetters   
parent: IsRich   attribute: HasScholarship   falseChild: falseLeaf   trueChild: GoodSAT   
parent: HasScholarship   -
parent: HasScholarship   attribute: GoodSAT   falseChild: falseLeaf   trueChild: GoodLetters   
parent: GoodSAT   -
parent: GoodSAT   attribute: GoodLetters   falseChild: falseLeaf   trueChild: trueLeaf   
parent: GoodLetters   -
parent: GoodLetters   -
parent: IsRich   attribute: GoodLetters   falseChild: SchoolActivities   trueChild: trueLeaf   
parent: GoodLetters   attribute: SchoolActivities   falseChild: falseLeaf   trueChild: trueLeaf   
parent: SchoolActivities   -
parent: SchoolActivities   -
parent: GoodLetters   -


Percentage of test cases correctly classified by a decision tree built with ID3 =  86 %
Percentage of test cases correctly classified by prior probabilities from the training set =  48 %






Tria

In [17]:
############################################
#The given structure for printing the tree does not say the classification of the tree
############################################